In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import KMeans
from scipy import stats
from scipy.spatial import distance

In [11]:
# 1. load & filter data 
data0 = pd.read_csv('~/Desktop/data/netFlow-02-14-2017.csv', sep = ',')
data1 = data0[(((data0.srcAddr == '128.143.2.75') & (data0.srcPort == 80)) | ((data0.dstAddr =='128.143.2.75') & (data0.dstPort == 80)))]
data2 = data1[["bytes","pkts","dur","rate"]]
len(data2)

255

In [6]:
# 2. run ISO/LOF, take anomalies out

In [19]:
# ISO
clf = IsolationForest(max_samples=1000, contamination=0.05,random_state= 5, bootstrap=False)
clf.fit(data2)
y_pred = clf.predict(data2)
data2['ISO'] = y_pred
anomalies = data2[data2.ISO == -1]
len(anomalies)

/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/sklearn/ensemble/iforest.py:183: UserWarning: max_samples (1000) is greater than the total number of samples (255). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))
/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


13

In [ ]:
# LOF
clf = LocalOutlierFactor(n_neighbors=20)
y_pred = clf.fit_predict(data2)
data2['LOF'] = y_pred
anomalies = data2[data2.LOF == -1]
len(anomalies)

In [20]:
# 3. run KSE 
def euc_dis(p1,p2):
    a = p1.values
    b = p2.values
    c = np.vstack((a,b))
    dist = distance.pdist(c)
    return float(dist)
def KSE_Test(df, sample_size):
    # initiate two sample dfs, one distance matrix, and one kse score.
    # generate 2 random samples.
    sample1 = df.sample(n = sample_size, replace = False)
    sample2 = df.sample(n = sample_size, replace = False)
    kse_score = [] # empty list to store KSE scores
#   distance_matrix = np.matrix([[None] * int(len(df) * fraction) , 
#                               [None] * int(len(df) * fraction)])
#     distance_matrix = [[0 for x in range(len(sample1))] 
#                        for y in range(len(sample2))] 
    distance_matrix = np.zeros((sample_size, sample_size))
    # calculate distance from all points in sample1 to all points in sample2
    for i in range(0,sample_size):
        for j in range(0,sample_size):
            distance_matrix[i][j] = euc_dis(sample1.iloc[i,],sample2.iloc[j,])
    print("half way there")
    
    # calculate average KS-score for each instance in df.
    for i in range(0, len(df)):
        observed_instance = df.iloc[i,]
        distance_array = [0] * sample_size
        for j in range(0,sample_size):
            distance_array[j] = euc_dis(observed_instance,sample2.iloc[j,])
        sum = 0
        for k in range(0, sample_size):
            ks_score = stats.ks_2samp(distance_array, distance_matrix[k])[0]
            sum += ks_score
        average = sum/sample_size
        #kse_score[i] = average
        kse_score.append((i, average))
        if (i%1000 == 0):
            print('loop %s' % i)
    return kse_score 
kse_result = KSE_Test(anomalies, 5)

half way there
loop 0


In [23]:
# 4. get result
index_anom = [] 
for i in kse_result:
    if i[1] > 0.3:
        index_anom.append(i[0])
# final result is a data set of the anomalies of the anomalies
final = anomalies.iloc[index_anom]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]